In [137]:
import os
import pandas as pd
import numpy as np
import pickle
import gc

TEST_CSV = r'C:\Users\e0817820\Desktop\tokka\data\additional_test\add_test.csv'
ADDITIONAL_TRAIN_CSV = r'C:\Users\e0817820\Desktop\tokka\data\additional_lastpoint\add_lastpoint.csv'
MODELS_DIR = r'C:\Users\e0817820\Desktop\tokka\models'

In [138]:
use_additional_train = False

In [139]:
list_cryptocurrencies = ['AVAX', 'ADA', 'SOL', 'BNB', 'TRX', 'DOGE', 'LINK', 'XRP', 'BTC', 'ETH']

df_train = pd.read_csv(ADDITIONAL_TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)
sample = df_test[['log_return']]
real_target = []
for symbol in list_cryptocurrencies:
    real_target.append(df_test[df_test['symbol'] == symbol]['log_return'])
df_test = df_test.drop(columns=['log_return'])
lower_limit_test = df_test['timestamp'].min()
upper_limit_train = df_train['timestamp'].max()
lower_limit_test_datetime = pd.to_datetime(df_test['timestamp'].min(), unit='ms')
upper_limit_train_datetime = pd.to_datetime(df_train['timestamp'].max(), unit='ms')
# df_train.drop(columns=['Unnamed: 0'])

In [140]:
print(df_test.shape)
print(df_test.columns)
print(df_train.columns)

(143800, 12)
Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_volume',
       'taker_sell_volume', 'symbol'],
      dtype='object')
Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_volume',
       'taker_sell_volume', 'symbol', 'log_return'],
      dtype='object')


In [141]:
df_combined = pd.concat([df_train, df_test])
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'], unit='ms')
all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary
print(df_combined.columns)
expanded_df = pd.DataFrame()
for symbol in list_cryptocurrencies:
    temp_df = pd.DataFrame({'timestamp': all_timestamps})
    temp_df['symbol'] = symbol 
    temp_df = pd.merge(temp_df, df_combined[df_combined['symbol'] == symbol], on='timestamp', how='left', suffixes=('', '_drop'))
    temp_df.drop([col for col in temp_df if col.endswith('_drop')], axis=1, inplace=True)
    expanded_df = pd.concat([expanded_df, temp_df])

expanded_df = expanded_df.sort_values(by=['timestamp', 'symbol'], ascending=[True, True])
expanded_df['timestamp'] = (expanded_df['timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

df_filled = pd.concat([expanded_df, df_test])
df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
df_filled.reset_index(drop=True, inplace=True)
df_filled.head()
del df_combined

df_filled.rename(columns={'index': 'timestamp'}, inplace=True)
print(df_filled.columns)
# df_filled.drop(columns=['id'], inplace=True)

Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_volume',
       'taker_sell_volume', 'symbol', 'log_return'],
      dtype='object')


C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\180169962.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary
C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\180169962.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))


Index(['timestamp', 'symbol', 'Unnamed: 0', 'open', 'high', 'low', 'close',
       'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_volume',
       'taker_sell_volume', 'log_return'],
      dtype='object')


C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\180169962.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\180169962.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(method='linear'))
C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\180169962.py:17: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.interpolate(meth

In [142]:
print(df_filled.shape)

(215810, 13)


In [143]:
df_filled.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\test_filled.csv', index=False)

In [144]:
df_filled.head()

,timestamp,symbol,Unnamed: 0,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_volume,taker_sell_volume,log_return
0,1707782460000,ADA,1429.00000,0.561300,0.561700,0.560600,0.561700,183790.600000,103155.631370,168.000000,121601.300000,68265.235660,0.001601
1,1707782520000,ADA,1430.79875,0.561307,0.561707,0.560607,0.561707,183792.992042,103158.341631,168.006944,121606.690958,68269.351035,0.001601
2,1707782580000,ADA,1432.59750,0.561314,0.561714,0.560614,0.561713,183795.384083,103161.051893,168.013889,121612.081917,68273.466409,0.001601
3,1707782640000,ADA,1434.39625,0.561320,0.561720,0.560620,0.561720,183797.776125,103163.762154,168.020833,121617.472875,68277.581784,0.001601
4,1707782700000,ADA,1436.19500,0.561327,0.561727,0.560627,0.561727,183800.168167,103166.472415,168.027778,121622.863833,68281.697158,0.001601


In [ ]:
combined_df1 = pd.DataFrame()
combined_df1[df_filled.columns] = 0
for id in range(10):
    combined_df1 = combined_df1.merge(df_filled.loc[df_filled["symbol"] == list_cryptocurrencies[id], ['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])
print(combined_df1.isna().sum())
combined_df1 = combined_df1.drop(df_filled.columns.drop("timestamp"), axis=1)
combined_df1 = combined_df1.sort_values('timestamp', ascending=True)
combined_df1 = combined_df1.fillna(method='ffill')
combined_df1.drop_duplicates(subset=['timestamp'], inplace=True)

combined_df1 = combined_df1[combined_df1['timestamp'] > lower_limit_test - 400*60*1000]
display(combined_df1.head())

timestamp         0
symbol        22603
Unnamed: 0    22603
open          22603
high          22603
              ...  
open_ETH          0
high_ETH          0
low_ETH           0
close_ETH         0
volume_ETH        0
Length: 63, dtype: int64


C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\1470829341.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df1 = combined_df1.fillna(method='ffill')


,timestamp,open_AVAX,high_AVAX,low_AVAX,close_AVAX,volume_AVAX,open_ADA,high_ADA,low_ADA,close_ADA,...,open_BTC,high_BTC,low_BTC,close_BTC,volume_BTC,open_ETH,high_ETH,low_ETH,close_ETH,volume_ETH
6801,1708190520000,39.820379,39.839271,39.807054,39.818162,1168.107675,0.607396,0.607796,0.607074,0.607512,...,51564.165496,51564.174942,51536.557029,51541.589587,18.091190,2779.837908,2780.054033,2778.503296,2778.891237,215.845465
6802,1708190580000,39.820203,39.839097,39.806886,39.817992,1168.108683,0.607402,0.607802,0.607080,0.607519,...,51564.404881,51564.414328,51536.796836,51541.827508,18.088578,2779.854706,2780.070289,2778.520481,2778.907475,215.604502
6803,1708190640000,39.820026,39.838924,39.806718,39.817821,1168.109692,0.607409,0.607809,0.607087,0.607526,...,51564.644265,51564.653714,51537.036643,51542.065429,18.085965,2779.871503,2780.086544,2778.537665,2778.923712,215.363539
6804,1708190700000,39.819850,39.838750,39.806550,39.817650,1168.110700,0.607416,0.607816,0.607094,0.607532,...,51564.883650,51564.893100,51537.276450,51542.303350,18.083352,2779.888300,2780.102800,2778.554850,2778.939950,215.122576
6805,1708190760000,39.819674,39.838576,39.806382,39.817479,1168.111708,0.607423,0.607823,0.607101,0.607539,...,51565.123035,51565.132486,51537.516257,51542.541271,18.080740,2779.905097,2780.119056,2778.572035,2778.956187,214.881612


KeyboardInterrupt: 

In [ ]:
combined_df2 = pd.DataFrame()
combined_df2[df_filled.columns] = 0
for id in range(10):
    combined_df2 = combined_df2.merge(df_filled.loc[df_filled["symbol"] == list_cryptocurrencies[id], ['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])
combined_df2 = combined_df2.drop(df_filled.columns.drop("timestamp"), axis=1)
combined_df2 = combined_df2.sort_values('timestamp', ascending=True)
combined_df2 = combined_df2.fillna(method='ffill')
combined_df2.drop_duplicates(subset=['timestamp'], inplace=True)

combined_df2 = combined_df2[combined_df2['timestamp'] > lower_limit_test - 400*60*1000]
display(combined_df2.head())

C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\628046445.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df2 = combined_df2.fillna(method='ffill')


,timestamp,open_AVAX,high_AVAX,low_AVAX,close_AVAX,volume_AVAX,open_ADA,high_ADA,low_ADA,close_ADA,...,open_BTC,high_BTC,low_BTC,close_BTC,volume_BTC,open_ETH,high_ETH,low_ETH,close_ETH,volume_ETH
6801,1708190520000,39.820379,39.839271,39.807054,39.818162,1168.107675,0.607396,0.607796,0.607074,0.607512,...,51564.165496,51564.174942,51536.557029,51541.589587,18.091190,2779.837908,2780.054033,2778.503296,2778.891237,215.845465
6802,1708190580000,39.820203,39.839097,39.806886,39.817992,1168.108683,0.607402,0.607802,0.607080,0.607519,...,51564.404881,51564.414328,51536.796836,51541.827508,18.088578,2779.854706,2780.070289,2778.520481,2778.907475,215.604502
6803,1708190640000,39.820026,39.838924,39.806718,39.817821,1168.109692,0.607409,0.607809,0.607087,0.607526,...,51564.644265,51564.653714,51537.036643,51542.065429,18.085965,2779.871503,2780.086544,2778.537665,2778.923712,215.363539
6804,1708190700000,39.819850,39.838750,39.806550,39.817650,1168.110700,0.607416,0.607816,0.607094,0.607532,...,51564.883650,51564.893100,51537.276450,51542.303350,18.083352,2779.888300,2780.102800,2778.554850,2778.939950,215.122576
6805,1708190760000,39.819674,39.838576,39.806382,39.817479,1168.111708,0.607423,0.607823,0.607101,0.607539,...,51565.123035,51565.132486,51537.516257,51542.541271,18.080740,2779.905097,2780.119056,2778.572035,2778.956187,214.881612


In [ ]:
print(combined_df1.shape)
print(combined_df2.shape)

(14779, 51)
(14779, 51)


In [ ]:
def bollinger_mavg(close, window=20):
    """Calculate the middle Bollinger Band."""
    return close.rolling(window=window).mean()

def bollinger_hband(close, window=20, k=2):
    """Calculate the upper Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg + (mstd * k)

def bollinger_lband(close, window=20, k=2):
    """Calculate the lower Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg - (mstd * k)

def bollinger_wband(close, window=20, k=2):
    """Calculate the width of the Bollinger Bands."""
    hband = bollinger_hband(close, window, k)
    lband = bollinger_lband(close, window, k)
    mavg = bollinger_mavg(close, window)
    return ((hband - lband) / mavg) * 100

def exponential_moving_average(close, span=20):
    """Calculate the Exponential Moving Average (EMA)."""
    return close.ewm(span=span, adjust=False).mean()

def relative_strength_index(close, window=14):
    """Calculate the Relative Strength Index (RSI)."""
    delta = close.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi
  
def stochastic_oscillator(high, low, close, window=14):
    """Calculate the Stochastic Oscillator."""
    low_min = low.rolling(window=window).min()
    high_max = high.rolling(window=window).max()
    k = ((close - low_min) / (high_max - low_min)) * 100
    return k

def williams_r(high, low, close, window=14):
    """Calculate Williams %R."""
    high_max = high.rolling(window=window).max()
    low_min = low.rolling(window=window).min()
    r = -100 * ((high_max - close) / (high_max - low_min))
    return r
  
def rate_of_change(close, window=14):
    """Calculate the Rate of Change (ROC)."""
    roc = ((close - close.shift(window)) / close.shift(window)) * 100
    return roc

In [ ]:
def delay(x, d):
    """Value of x d days ago"""
    return x.shift(d)

def correlation(x, y, d):
    """Time-series correlation of x and y for the past d days"""
    return x.rolling(window=d).corr(y)

def delta(x, d):
    """Today's value of x minus the value of x d days ago"""
    return x.diff(d)

def ts_max(x, d):
    """Time-series max over the past d days"""
    return x.rolling(window=d).max()

def ts_min(x, d):
    """Time-series min over the past d days"""
    return x.rolling(window=d).min()

In [ ]:
def getFeaturesVer1(df, limit):
    lags_normal = [15, 40, 150, 300]
    lags_ema = [15, 50, 160, 300]
    lags_rsi = [15, 50, 160, 300]
    lags_bbands = [15, 50, 160, 300]
    for id in range(10):   
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_bbands:
            # df[f'hbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_hband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'lbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_lband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'width_bbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'hbbands_close_{lag}_{symbol}'] = bollinger_hband(df[f'close_{symbol}'], window=lag)
            df[f'lbbands_close_{lag}_{symbol}'] = bollinger_lband(df[f'close_{symbol}'], window=lag)
            df[f'width_bbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag)
        # Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1))) 
        df['alpha12_'+symbol] = np.sign(delta(df['volume_'+symbol], 1)) * (-1 * delta(df['close_'+symbol], 1))
        # Alpha#54: ((-1 * ((low - close) * (open^5))) / (0.0000001 + (low - high) * (close^5))) 
        df['alpha54_'+symbol] = ((-1 * ((df['low_'+symbol] - df['close_'+symbol]) * (df['open_'+symbol]**5))) / (0.0000001 + (df['low_'+symbol] - df['high_'+symbol]) * (df['close_'+symbol]**5)))
        # Alpha#41: (((high * low)^0.5) - close)
        # df['alpha41_'+symbol] = ((df['high_'+symbol] * df['low_'+symbol])**0.5) - df['close_'+symbol]
        # Alpha#53: (-1 * delta((((close - low) - (high - close)) / (close - low + 0.0000001)), 9))
        # df['alpha53_'+symbol] = -1 * delta((((df['close_'+symbol] - df['low_'+symbol]) - (df['high_'+symbol] - df['close_'+symbol])) / (df['close_'+symbol] - df['low_'+symbol] + 0.0000001)), 9)
        # Alpha#6: (-1 * correlation(open, volume, 10)) 
        # df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
        # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
        df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_'+symbol], 1)))

    gc.collect()
    df.fillna(method='ffill', inplace=True)
    # df.fillna(method='bfill', inplace=True)  
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'open_{symbol}', f'high_{symbol}', f'low_{symbol}', f'volume_{symbol}'], inplace=True)
    gc.collect()  
    df = df[df['timestamp'] >= limit] 
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    return df   

In [ ]:
def getFeaturesVer2(df, limit):
    lags_normal = [8, 20, 120]
    lags_ema = [15]
    lags_rsi = [8, 14]
    lags_stoch_osci = [10, 25]
    roc_window = [8, 15]
    williams_percentage_r = [10, 20]
    lags_wbands = [10, 50]
    for id in range(10):        
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_stoch_osci:
            # df[f'stoch_oscil_close_{lag}_{symbol}'] = ta.momentum.stoch(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'stoch_oscil_close_{lag}_{symbol}'] = stochastic_oscillator(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in roc_window:
            # df[f'roc_close_{lag}_{symbol}'] = ta.momentum.roc(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'roc_close_{lag}_{symbol}'] = rate_of_change(df[f'close_{symbol}'], window=lag)
        for lag in williams_percentage_r:
            # df[f'williams_close_{lag}_{symbol}'] = ta.momentum.williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], lbp=lag, fillna=False)  
            df[f'williams_close_{lag}_{symbol}'] = williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in lags_wbands:
            # df[f'wbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'wbbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag) 
        # Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1)))
        df['alpha12_'+symbol] = np.sign(delta(df['volume_'+symbol], 1)) * (-1 * delta(df['close_'+symbol], 1))
        # Alpha#41: (((high * low)^0.5) - close)
        df['alpha41_'+symbol] = ((df['high_'+symbol] * df['low_'+symbol])**0.5) - df['close_'+symbol]
        # Alpha#53: (-1 * delta((((close - low) - (high - close)) / (close - low + 0.0000001)), 9))
        df['alpha53_'+symbol] = -1 * delta((((df['close_'+symbol] - df['low_'+symbol]) - (df['high_'+symbol] - df['close_'+symbol])) / (df['close_'+symbol] - df['low_'+symbol] + 0.0000001)), 9)
        # Alpha#6: (-1 * correlation(open, volume, 10)) 
        df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
        # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
        df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_'+symbol], 1)))
    gc.collect()  
    # df.dropna(inplace=True)
    df.fillna(method='ffill', inplace=True)
    # df.fillna(method='bfill', inplace=True)
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'high_{symbol}', f'low_{symbol}', f'open_{symbol}', f'volume_{symbol}'], inplace=True)
    gc.collect()  
    df = df[df['timestamp'] >= limit] 
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    return df   

In [ ]:
combined_df1 = getFeaturesVer1(combined_df1, lower_limit_test)
print(combined_df1.shape)
combined_df2 = getFeaturesVer2(combined_df2, lower_limit_test)
print(combined_df2.shape)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\2746239995.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\2746239995.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
C

(14380, 409)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\3820758813.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\3820758813.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['clos

(14380, 297)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\3820758813.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['timestamp'], inplace=True)


In [ ]:
num_models = 12

In [ ]:
def correlation(a, b):    
    a = np.ravel(a)
    b = np.ravel(b)

    len_data = len(a)
    mean_a = np.mean(a)
    mean_b = np.mean(b)
    var_a = np.var(a, ddof=0)  
    var_b = np.var(b, ddof=0)  
    
    cov = np.sum((a - mean_a) * (b - mean_b)) / len_data
    corr = cov / np.sqrt(var_a * var_b)

    return corr

In [ ]:
x = 4

In [ ]:
def predictForEachAssetVer1(df):
    preds = []
    output = pd.DataFrame()
    for id, symbol in enumerate(list_cryptocurrencies):
        
        pred_asset = []
        symbol = list_cryptocurrencies[id]
        target = f'log_return_{symbol}'
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        df_asset = df[columns]
        
        #load models
        models = []
        for fold in range(num_models):
            model = pickle.load(open(os.path.join(MODELS_DIR, f'{symbol}_fold{fold}_1.pkl'), 'rb'))
            models.append(model)
        df_asset = df_asset.fillna(method='ffill')
        df_asset = df_asset.drop(columns=[target, 'timestamp']) 
        corr_list = []
        for model in models: 
            pred_fold = model.predict(df_asset)
            corr_list.append(correlation(pred_fold, real_target[id]))
            pred_asset.append(pred_fold)
        print(f'{symbol} corr: {corr_list}')
        print(f'{symbol} mean corr: {np.mean(corr_list)}')

        pred = np.mean(pred_asset, axis=0)
        del models
        preds.append(pred)
        output_symbol = pd.DataFrame({'timestamp': df['timestamp'], 'symbol': symbol, 'log_return': pred})
        output_symbol.fillna(method='ffill', inplace=True)
        output = pd.concat([output, output_symbol])
        gc.collect()
    return output

In [ ]:

def predictForEachAssetVer2(df):
    preds = []
    output = pd.DataFrame()
    for id, symbol in enumerate(list_cryptocurrencies):
        
        pred_asset = []
        target = f'log_return_{symbol}'
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        df_asset = df[columns]
        
        #load models
        models = []
        for fold in range(num_models):
            # if fold in [0]:
                # fold = 89
                model = pickle.load(open(os.path.join(MODELS_DIR, f'{symbol}_fold{fold}_2.pkl'), 'rb'))
                models.append(model)
                # break
        df_asset = df_asset.fillna(method='ffill')
        df_asset = df_asset.drop(columns=[target, 'timestamp']) 
        corr_list = []
        for model in models: 
            pred_fold = model.predict(df_asset)
            corr_list.append(correlation(pred_fold, real_target[id]))
            pred_asset.append(pred_fold)
        print(f'{symbol} corr: {corr_list}') 
        print(f'{symbol} mean corr: {np.mean(corr_list)}')
        pred = np.mean(pred_asset, axis=0)
        del models
        preds.append(pred)
        output_symbol = pd.DataFrame({'timestamp': df['timestamp'], 'symbol': symbol, 'log_return': pred})
        output_symbol.fillna(method='ffill', inplace=True)
        output = pd.concat([output, output_symbol])
        gc.collect()
    return output

In [ ]:
output1 = predictForEachAssetVer1(combined_df1)
print(output1.shape)
output2 = predictForEachAssetVer2(combined_df2)
print(output2.shape)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_31648\2510973699.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')


KeyError: "['log_return_AVAX'] not found in axis"

In [ ]:
idx = output1[['timestamp', 'symbol']]
log_return1 = output1[['log_return']]
log_return2 = output2[['log_return']]
max_corr = 0
max_k = 0
for k in np.arange(0, 1.05, 0.05):
    print(f"Ver1 coefficient: {k}")
    print(f"Ver2 coefficient: {1-k}")
    log_return_mean = (k*log_return1 + (1-k)*log_return2)
    output = pd.concat([idx, log_return_mean], axis=1)
    output = df_test.merge(output, on=['timestamp', 'symbol'], how='left')[['log_return', 'timestamp', 'symbol']]
    corr_list = []
    for symbol in list_cryptocurrencies:
        pred = output[output['symbol'] == symbol][['log_return']]
        real = sample[output['symbol'] == symbol][['log_return']]
        corr_list.append(correlation(pred, real))
    print(corr_list)
    print(f"Corr score: {np.mean(corr_list)}")
    if (np.mean(corr_list) > max_corr):
        max_corr = np.mean(corr_list)
        max_k = k
print(f"Max_corr: {max_corr}, Max_k: {max_k}")

Ver1 coefficient: 0.0
Ver2 coefficient: 1.0
[0.02030372969734276, 0.033846843465521095, 0.04094023325411494, 0.013862927354550993, 0.012376162860564777, 0.08960280464727616, 0.061300340469642285, 0.03163566351669609, 0.0307946586069319, -0.04221737074425557]
Corr score: 0.029244599312838543
Ver1 coefficient: 0.05
Ver2 coefficient: 0.95
[0.01985898781264974, 0.03415410934308348, 0.040627657061937356, 0.013510247682791833, 0.010794530339070126, 0.09005297318471248, 0.06189000365813974, 0.0321034201892235, 0.03261655197378902, -0.04293034108782084]
Corr score: 0.02926781401575764
Ver1 coefficient: 0.1
Ver2 coefficient: 0.9
[0.0193959020048611, 0.03444812585906634, 0.04027509771776917, 0.013146228246851526, 0.009140983785173206, 0.09045133225914809, 0.062465960013018, 0.03255455144473178, 0.03450731660428268, -0.04364129131014515]
Corr score: 0.02927442066247568
Ver1 coefficient: 0.15000000000000002
Ver2 coefficient: 0.85
[0.018914565615451184, 0.03472760484107483, 0.039882330214418844, 0.

In [ ]:
print(output.shape)

(143800, 3)
